### recursos geospark - sedona

https://jiayuasu.github.io/files/paper/GeoSpark_Geoinformatica_2018.pdf

https://sci-hub.se/10.1145/3221269.3223040


In [0]:
%ls -la '/dbfs/FileStore/tables/Catastro/'
#%ls -la '/dbfs/FileStore/tables/procesado/'

In [0]:
from pyspark.sql import SparkSession
from pyspark import StorageLevel
from sedona.spark import *
from sedona.utils import SedonaKryoRegistrator, KryoSerializer

from sedona.register.geo_registrator import SedonaRegistrator

# configuración de spark y Sedona para datos geograficos
spark = SparkSession.\
    builder.\
    master("local[*]").\
    appName("Sedona App").\
    config("spark.serializer", KryoSerializer.getName).\
    config("spark.kryo.registrator", SedonaKryoRegistrator.getName) .\
    config("spark.kryoserializer.buffer.max", "150000m").\
    getOrCreate()
    #config("sedona.global.charset", "utf8").\
    
#SedonaRegistrator.registerAll()
SedonaContext.create(spark)

In [0]:
%sql
select count(*) from lotes_catastro_2006_2023;

In [0]:
import geopandas as gpd
# se importa la base de datos compilada historica de catastro, la cual se encuentra en el notebook, tableros. 
# pendiente hacer merge de las ramas en github.
# compilado catastro alexander con modificaciones a octubre.
#path = 'dbfs:/FileStore/df/compiladoCatastro.csv'
path = '/FileStore/tables/Catastro/union_catastro_2006_2023_file.csv'
df = spark.read.format("csv").option("header", "true").load(path)

In [0]:
from sedona.sql.st_constructors import ST_Point
from pyspark.sql.functions import col, cast
from pyspark.sql.types import DoubleType, StringType
# trandformación de tipo de dato y se renombran las variables.
df = df.withColumn("lng", col("DefLng").cast("double"))
df = df.withColumn("lat", col("DefLat").cast("double"))
# generación de la estructura wkt como punto(x,y), para ser leida como un estandar posteriormente
pointcatastro_df = df.withColumn("wkt", ST_Point(df["lng"],df["lat"]).cast(StringType()))
# pasa a pandas, por facilidad.
point = pointcatastro_df.toPandas()

In [0]:
point[point['AnioVigencia']=='2023']

In [0]:
# se crea un geodataframe para operarlo geograficamente.
geo_point = gpd.GeoDataFrame(point, geometry=gpd.points_from_xy(point.lng, point.lat), crs = "EPSG:4326")

In [0]:
geo_point.shape

In [0]:
#ADQUISICION: predios adquiridos para la primera linea del metro.
# AREA PLMB
adquisicion = gpd.GeoDataFrame.from_file('/dbfs/FileStore/shared_uploads/duvan.robles@metrodebogota.gov.co/predios.shp').to_crs('epsg:4326')
Entropia=gpd.overlay(geo_point, adquisicion, how='difference', keep_geom_type=False)
Entropia.shape

In [0]:
Entropia['AC'] = Entropia['AC'].astype('float')
Entropia['AT'] = Entropia['AT'].astype('float')

In [0]:
# borrando 2023!!!!!!!!!!
Entropia.drop(Entropia[Entropia.AnioVigencia == '2023'].index, inplace=True)

In [0]:
#copia del dataframe para operar, solo se hace el copy por efecto de ser practicos.
copy_Entropia = Entropia.copy()
# cambio de tipo de datos, es posible que sean de tipo object y no se operen correctamente, por esta razon se convierten en enteros.
copy_Entropia = copy_Entropia.astype({'AsociarAlUsoPrincipal':'int', 'ComercioYServicios':'int', 'Dotacional':'int',
       'Industrial':'int', 'Otro':'int', 'Residencial':'int', 'UsoSinAsociar':'int'})

def usoFinal(row):
    lista_nombres = ['Asociar Al Uso Principal','Comercio Y Servicios','Dotacional','Industrial','Otro','Residencial', 'Uso Sin Asociar']
    lista = [row['AsociarAlUsoPrincipal'],row['ComercioYServicios'],row['Dotacional'],row['Industrial'],row['Otro'],row['Residencial'],row['UsoSinAsociar']]
    maximo = max(lista)
    if maximo == 0:
        return "Uso no reportado"
    else:
       #  if lista.index(maximo)==6:
       #      return "Uso no reportado"
        if lista.index(maximo)==0:
            listaAsignacion_nombres = ['Comercio Y Servicios','Dotacional','Industrial','Otro','Residencial', 'Uso Sin Asociar']
            #'Comercio Y Servicios','Dotacional','Industrial','Otro','Residencial'
            listaAsignacion = [row['ComercioYServicios'],row['Dotacional'],row['Industrial'],row['Otro'],row['Residencial'],row['UsoSinAsociar']]
            asignacion = max(listaAsignacion)
            return listaAsignacion_nombres[listaAsignacion.index(asignacion)]
        else:
            return lista_nombres[lista.index(maximo)]
 
copy_Entropia['usoPrincipal'] = copy_Entropia.apply(usoFinal, axis=1)
copy_Entropia = copy_Entropia.drop(copy_Entropia[copy_Entropia['usoPrincipal']=='Uso no reportado'].index)

In [0]:
copy_Entropia = copy_Entropia.drop(copy_Entropia[copy_Entropia['usoPrincipal']=='Uso Sin Asociar'].index)

In [0]:
copy_Entropia['usoPrincipal'].value_counts()

In [0]:
import pandas as pd
pd.pivot_table(copy_Entropia, values='Estacion', index=['AnioVigencia', 'usoPrincipal'],
                    aggfunc="count").unstack()

In [0]:
AGRUPADOS = copy_Entropia.groupby(by=['Linea','Estacion','Tiempo','AnioVigencia'])['AC','AT'].sum().reset_index()

In [0]:
copy_Entropia['usoPrincipal'].value_counts()

In [0]:
AGRUPADOS_ZONA = copy_Entropia.groupby(by=['Linea','Estacion','Tiempo', 'usoPrincipal', 'AnioVigencia',])['AC','AT'].sum().reset_index()

In [0]:
AGRUPADOS.shape
AGRUPADOS_ZONA.shape

In [0]:
import pandas as pd
Pre_Entropia = pd.merge(
    AGRUPADOS_ZONA, AGRUPADOS,
    how='inner',
    left_on=['Linea','Estacion','Tiempo','AnioVigencia'],
    right_on=['Linea','Estacion','Tiempo','AnioVigencia']).rename(
        columns={'AC_y':'Construida_total','AT_y':'Terreno_total',
                 'AC_x':'AC_Uso','AT_x':'AT_Uso'})

In [0]:
Pre_Entropia['usoPrincipal'].value_counts()

In [0]:
import numpy as np
Pre_Entropia['factor'] = Pre_Entropia['AC_Uso']/Pre_Entropia['Construida_total']
Pre_Entropia['ln_factor'] = np.log(Pre_Entropia['AC_Uso']/Pre_Entropia['Construida_total'])
Pre_Entropia['MULT'] = Pre_Entropia['factor'] * Pre_Entropia['ln_factor']

In [0]:
Entropia_Final = Pre_Entropia.groupby(by=['Linea','Estacion','Tiempo','AnioVigencia']).agg({'MULT': 'sum'})
Entropia_Final['ValEntropia_estacion'] = -Entropia_Final['MULT']/np.log(len(set(Pre_Entropia.usoPrincipal.values.tolist())))

In [0]:
Entropia_Final = Entropia_Final.reset_index()

In [0]:
#
Entropia_Final['AnioVigencia'] = Entropia_Final['AnioVigencia'].astype('int')
#ef = Entropia_Final.drop(Entropia_Final[(Entropia_Final['AnioVigencia']==0) | (Entropia_Final['AnioVigencia']==2023)].index)
#& (Entropia_Final['AnioVigencia']==2023)]


In [0]:
Entropia_Final.tail(50)

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

ef = Entropia_Final[Entropia_Final['Linea']=='Linea 1']
# Linea 1

Linea = 'PLMB'
#for xanio in range(2006,2023):

sns.set_theme(style="whitegrid")

plt.figure(figsize=(20,10))
plt.xticks(rotation=40)
#lista = list(compilado[compilado.layer.isin(['E16'])].Uso_Agrupado.value_counts().head(10).to_dict().keys())
# compila por estacion
#table = pd.pivot_table(compilado[(compilado.layer.isin([e]) & compilado.Uso_Agrupado.isin(Usos_analisis) & compilado.Tiempo.isin(['5 min','10 min']))] , values=['AT_VM2'], index=['PERIODO','Uso_Agrupado'],aggfunc={'AT_VM2': np.mean})
# compila el agregado
#table = pd.pivot_table(compilado[(compilado.Linea.isin(['Linea 1']) & compilado.Tiempo.isin(['5 min','10 min']))] , values=['AC_VM2'], index=['PERIODO','Uso_Agrupado'],aggfunc={'AC_VM2': np.mean})

ax = sns.lineplot(data=ef, x="AnioVigencia", y="ValEntropia_estacion", hue="Estacion", style="Estacion", markers=True ,palette="tab20")

#sns.lineplot(data=F[F.layer_1.isin(Estaciones_seg2)], x="PERIODO", y="Valor Entropia estacion", hue="NOMBRE", style="NOMBRE", markers=True ,palette="tab20")

ax.set_title('{} - ENTROPIA TOTAL USOS -- {}'.format(Linea, sorted(list(set(Pre_Entropia.usoPrincipal.values.tolist()))) ).upper(),pad=10)
ax.set_xlabel('Periodos Catastro'.upper())
ax.set_ylabel('ENTROPIA TOTAL'.upper())

plt.legend(title='Estación', loc='upper left', bbox_to_anchor=(1,1))
plt.tight_layout()
ax.yaxis.labelpad = 20
ax.xaxis.labelpad = 20
ax.tick_params(labelsize=12)
ax.grid(alpha=0.5, linestyle='dashed', linewidth=1)
ax.spines['bottom'].set_color('none')
ax.spines['top'].set_color('none')
ax.spines['left'].set_color('none')
ax.spines['right'].set_color("none")

In [0]:
entropia_pdf = spark.createDataFrame(Entropia_Final)
entropia_pdf.createOrReplaceTempView("ENTROPIA_TOTAL_v2")

In [0]:
entropia_pdf

In [0]:
%sql
CREATE OR REPLACE TABLE ent_total_v2 AS (
  Select 
  * 
  from 
  ENTROPIA_TOTAL_v2);

In [0]:
%sql
Select  *  from  ent_total_v2;

### Calculo entropia banca, solo servicios y residencial.

In [0]:
EntBanca = copy_Entropia[(copy_Entropia['usoPrincipal'] == 'Comercio Y Servicios') | (copy_Entropia['usoPrincipal'] == 'Residencial')].copy()
# Comercio Y Servicios	Dotacional	Industrial	Otro	Residencial

In [0]:
EntBanca['usoPrincipal'].value_counts()

In [0]:
EntBanca.columns

In [0]:
import pandas as pd
pd.pivot_table(EntBanca[(EntBanca['Estacion']=='E13') & (EntBanca['Tiempo']=='5 min')], values='Estacion', index=['AnioVigencia', 'usoPrincipal'],
                    aggfunc="count").unstack()

In [0]:
EntBanca['AC'] = EntBanca['AC'].astype('float')
EntBanca['AT'] = EntBanca['AT'].astype('float')
AGRUPADOS_Banca = EntBanca.groupby(by=['Linea','Estacion','Tiempo','AnioVigencia'])['AC','AT'].sum().reset_index()

Agrupación.

In [0]:
AGRUPADOS_ZONA_Banca = EntBanca.groupby(by=['Linea','Estacion','Tiempo', 'usoPrincipal', 'AnioVigencia',])['AC','AT'].sum().reset_index()

In [0]:
import pandas as pd
Pre_Banca = pd.merge(
    AGRUPADOS_ZONA_Banca, AGRUPADOS_Banca,
    how='inner',
    left_on=['Linea','Estacion','Tiempo','AnioVigencia'],
    right_on=['Linea','Estacion','Tiempo','AnioVigencia']).rename(
        columns={'AC_y':'Construida_total','AT_y':'Terreno_total',
                 'AC_x':'AC_Uso','AT_x':'AT_Uso'})

In [0]:
import numpy as np
Pre_Banca['factor'] = Pre_Banca['AC_Uso']/Pre_Banca['Construida_total']
Pre_Banca['ln_factor'] = np.log(Pre_Banca['AC_Uso']/Pre_Banca['Construida_total'])
Pre_Banca['MULT'] = Pre_Banca['factor'] * Pre_Banca['ln_factor']

In [0]:
Pre_Banca[(Pre_Banca['Estacion']=='E13') & (Pre_Banca['Tiempo']=='5 min')]

In [0]:
Banca_Final = Pre_Banca.groupby(by=['Linea','Estacion','Tiempo','AnioVigencia']).agg({'MULT': 'sum'})
Banca_Final['ValEntropia_estacion'] = -Banca_Final['MULT']/np.log(len(set(Pre_Banca.usoPrincipal.values.tolist())))

In [0]:
e=Banca_Final.reset_index()
e[(e['Estacion']=='E13') & (e['Tiempo']=='5 min')]

In [0]:
Banca_Final[(Banca_Final['Estacion']=='E13') & (Banca_Final['Tiempo']=='5 min')]

In [0]:
Banca_Final = Banca_Final.reset_index()

In [0]:
Banca_Final['AnioVigencia'] = Banca_Final['AnioVigencia'].astype('int')
ef_banca = Banca_Final
#.drop(Banca_Final[(Banca_Final['AnioVigencia']==0) | (Banca_Final['AnioVigencia']==2023)].index)
ef_banca

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Linea 1

Linea = 'PLMB'
#for xanio in range(2006,2023):

sns.set_theme(style="whitegrid")

plt.figure(figsize=(20,10))
plt.xticks(rotation=40)
#lista = list(compilado[compilado.layer.isin(['E16'])].Uso_Agrupado.value_counts().head(10).to_dict().keys())
# compila por estacion
#table = pd.pivot_table(compilado[(compilado.layer.isin([e]) & compilado.Uso_Agrupado.isin(Usos_analisis) & compilado.Tiempo.isin(['5 min','10 min']))] , values=['AT_VM2'], index=['PERIODO','Uso_Agrupado'],aggfunc={'AT_VM2': np.mean})
# compila el agregado
#table = pd.pivot_table(compilado[(compilado.Linea.isin(['Linea 1']) & compilado.Tiempo.isin(['5 min','10 min']))] , values=['AC_VM2'], index=['PERIODO','Uso_Agrupado'],aggfunc={'AC_VM2': np.mean})

ax = sns.lineplot(data=ef_banca[(ef_banca['Linea'] == 'Linea 1')], x="AnioVigencia", y="ValEntropia_estacion", hue="Estacion", style="Estacion", markers=True ,palette="tab20")

#sns.lineplot(data=F[F.layer_1.isin(Estaciones_seg2)], x="PERIODO", y="Valor Entropia estacion", hue="NOMBRE", style="NOMBRE", markers=True ,palette="tab20")

ax.set_title('{} - ENTROPIA TOTAL USOS -- {}'.format(Linea, sorted(list(set(EntBanca.usoPrincipal.values.tolist()))) ).upper(),pad=10)
ax.set_xlabel('Periodos Catastro'.upper())
ax.set_ylabel('ENTROPIA TOTAL'.upper())

plt.legend(title='Estación', loc='upper left', bbox_to_anchor=(1,1))
plt.tight_layout()
ax.yaxis.labelpad = 20
ax.xaxis.labelpad = 20
ax.tick_params(labelsize=12)
ax.grid(alpha=0.5, linestyle='dashed', linewidth=1)
ax.spines['bottom'].set_color('none')
ax.spines['top'].set_color('none')
ax.spines['left'].set_color('none')
ax.spines['right'].set_color("none")

In [0]:
entBanca_pdf = spark.createDataFrame(ef_banca)
entBanca_pdf.createOrReplaceTempView("ENTBANCA_TOTAL_V2")

In [0]:
entBanca_pdf.display()

In [0]:
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="whitegrid")
df1 = entBanca_pdf.toPandas()
df2 = pd.pivot_table(df1[df1['Linea']=='Linea 1'], values=['ValEntropia_estacion'], index=['Estacion', 'AnioVigencia']).unstack()

#df2.set_index('NOMBRE', inplace = True)
#sns.heatmap(df2, linewidth=.5)

plt.figure(figsize=(25,10))
plt.xticks(rotation=40)
cmap = sns.diverging_palette(230, 20, sep=70 , as_cmap=True)
sns.heatmap(df2, cmap=cmap, annot=True, fmt=".3f", linewidths=.5)

In [0]:
%sql
CREATE OR REPLACE TABLE ent_banca_v2 AS (
  Select 
  * 
  from 
  ENTBANCA_TOTAL_V2);

In [0]:
%sql
Select  *  from  ent_banca_v2 where Estacion=='E13' and Tiempo=='5 min';

In [0]:
%sql
Select * from ent_total;

In [0]:
cases = [
    F.when(F.col("AsociarAlUsoPrincipal") == F.greatest(*columns_to_compare), 
           F.when(F.col("ComercioYServicios") == F.greatest(*[col for col in columns_to_compare if col != "AsociarAlUsoPrincipal"]), "ComercioYServicios")
           .when(F.col("Dotacional") == F.greatest(*[col for col in columns_to_compare if col != "AsociarAlUsoPrincipal"]), "Dotacional")
           .when(F.col("Industrial") == F.greatest(*[col for col in columns_to_compare if col != "AsociarAlUsoPrincipal"]), "Industrial")
           .when(F.col("Otro") == F.greatest(*[col for col in columns_to_compare if col != "AsociarAlUsoPrincipal"]), "Otro")
           .when(F.col("Residencial") == F.greatest(*[col for col in columns_to_compare if col != "AsociarAlUsoPrincipal"]), "Residencial")
           .when(F.col("UsoSinAsociar") == F.greatest(*[col for col in columns_to_compare if col != "AsociarAlUsoPrincipal"]), "UsoSinAsociar")
           .otherwise(None)
          )
]
